# Validation of the 3th, 4th and 6th Order VBR SynGen with Controllers (Exciter and Turbine Governor) against PSAT

## 1. Run C++ examples with Exciter

In [ ]:
import os
import subprocess

#%matplotlib widget

name = 'SP_ReducedOrderSG_SMIB_Fault'
options = ['SGModel=3', 'SGModel=4', 'SGModel=6a', 'SGModel=6b']

dpsim_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

path_exec = dpsim_path + '/build/dpsim/examples/cxx/'
for i in range(len(options)):
    sim = subprocess.Popen([path_exec + name, '-o', options[i], '-oWithExciter=true',  '-oFinalTime=10'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(sim.communicate()[0].decode())


## Load DPSIM Results

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os

work_dir = os.getcwd() + "/logs/"
file_name_3Order = 'SP_SynGen3Order_VBR_SMIB_Fault'
file_name_4Order = 'SP_SynGen4Order_VBR_SMIB_Fault'
file_name_6aOrder = 'SP_SynGen6aOrder_VBR_SMIB_Fault'
file_name_6bOrder = 'SP_SynGen6bOrder_VBR_SMIB_Fault'


path_logfile = work_dir + file_name_3Order + "/" + file_name_3Order + ".csv"
ts_dpsim_3Order = read_timeseries_dpsim(path_logfile)

path_logfile = work_dir + file_name_4Order + "/" + file_name_4Order + ".csv"
ts_dpsim_4Order = read_timeseries_dpsim(path_logfile)

path_logfile = work_dir + file_name_6aOrder + "/" + file_name_6aOrder + ".csv"
ts_dpsim_6aOrder = read_timeseries_dpsim(path_logfile)

path_logfile = work_dir + file_name_6bOrder + "/" + file_name_6bOrder + ".csv"
ts_dpsim_6bOrder = read_timeseries_dpsim(path_logfile)

## Load PSAT Results

In [ ]:
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/SMIB-Fault/PSAT_3OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.txt'
local_file_3Order = 'reference-results/PSAT_3OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.out'
urllib.request.urlretrieve(url, local_file_3Order) 
ts_psat_3Order = read_timeseries_dpsim(local_file_3Order)

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/SMIB-Fault/PSAT_4OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.txt'
local_file_4Order = 'reference-results/PSAT_4OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.out'
urllib.request.urlretrieve(url, local_file_4Order)
ts_psat_4Order = read_timeseries_dpsim(local_file_4Order)

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/SMIB-Fault/PSAT_6OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.txt'
local_file_6Order = 'reference-results/PSAT_6OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.out'
urllib.request.urlretrieve(url, local_file_6Order)
ts_psat_6Order = read_timeseries_dpsim(local_file_6Order)

## Calculate $V_d$, $V_q$, $I_d$ and $I_q$ in PSAT

### 3th Order Model

In [ ]:
#calculate vd and vq
angle = ts_psat_3Order['delta'].values - ts_psat_3Order['theta_bus1'].values
V_bus1 = ts_psat_3Order['V_Bus1'].values
vd = V_bus1 * np.sin(angle)
ts_psat_3Order['vd'] = TimeSeries('vd', ts_psat_3Order['V_Bus1'].time, vd)
vq = V_bus1 * np.cos(angle)
ts_psat_3Order['vq'] = TimeSeries('vq', ts_psat_3Order['V_Bus1'].time, vq)

#calculate id and iq in PSAT
Xd_t = 0.299900
Xq = 1.7600
i_d = (ts_psat_3Order['eq_t'].values - ts_psat_3Order['vq'].values) / Xd_t
ts_psat_3Order['id'] = TimeSeries('id', ts_psat_3Order['eq_t'].time, i_d)
i_q = ts_psat_3Order['vd'].values / Xq
ts_psat_3Order['iq'] = TimeSeries('iq', ts_psat_3Order['vd'].time, i_q)

### 4th Order Model

In [ ]:
#calculate vd and vq
angle = ts_psat_4Order['delta'].values - ts_psat_4Order['theta_bus1'].values
V_bus1 = ts_psat_4Order['V_Bus1'].values
vd = V_bus1 * np.sin(angle)
ts_psat_4Order['vd'] = TimeSeries('vd', ts_psat_4Order['V_Bus1'].time, vd)
vq = V_bus1 * np.cos(angle)
ts_psat_4Order['vq'] = TimeSeries('vq', ts_psat_4Order['V_Bus1'].time, vq)
    
#calculate id and iq in PSAT
Xd = 0.299900
Xq = 0.650000
i_d = (ts_psat_4Order['eq_t'].values - ts_psat_4Order['vq'].values) / Xd
ts_psat_4Order['id'] = TimeSeries('id', ts_psat_4Order['eq_t'].time, i_d)
i_q = -(ts_psat_4Order['ed_t'].values - ts_psat_4Order['vd'].values) / Xq
ts_psat_4Order['iq'] = TimeSeries('iq', ts_psat_4Order['ed_t'].time, i_q)

### 6th Order Model (Marconato's model)

In [ ]:
#calculate vd and vq
angle = ts_psat_6Order['delta'].values - ts_psat_6Order['theta_bus1'].values
V_bus1 = ts_psat_6Order['V_Bus1'].values
vd = V_bus1 * np.sin(angle)
ts_psat_6Order['vd'] = TimeSeries('vd', ts_psat_6Order['V_Bus1'].time, vd)
vq = V_bus1 * np.cos(angle)
ts_psat_6Order['vq'] = TimeSeries('vq', ts_psat_6Order['V_Bus1'].time, vq)

#calculate id and iq in PSAT\n",
Xd_s = 0.2299
Xq_s = 0.2500
i_d = (ts_psat_6Order['eq_s'].values - ts_psat_6Order['vq'].values) / Xd_s
ts_psat_6Order['id'] = TimeSeries('id', ts_psat_6Order['eq_s'].time, i_d)
i_q = -(ts_psat_6Order['ed_s'].values - ts_psat_6Order['vd'].values) / Xq_s
ts_psat_6Order['iq'] = TimeSeries('iq', ts_psat_6Order['ed_s'].time, i_q)

## Nominal values

In [ ]:
#nominal values (per unit)
nominal_values = {}
nominal_values['Te'] = 0.540541
nominal_values['Tm'] = 0.540541
nominal_values['Edq0_0'] = 0.403062     #4th order model
nominal_values['Edq0_1'] = 0.944802     #3th and 4th order model
nominal_values['Edq0_t_0'] = 0.392200   #6 order model
nominal_values['Edq0_t_1'] = 0.946405   #6 order model
nominal_values['Edq0_s_0'] = 0.548309   #6 order model
nominal_values['Edq0_s_1'] = 0.918731   #6 order model
nominal_values['Vdq0_0'] = 0.639089
nominal_values['Vdq0_1'] = 0.833106
nominal_values['Idq0_0'] = 0.372443
nominal_values['Idq0_1'] = 0.363119
nominal_values['delta'] = 0.909230
nominal_values['w_r'] = 1.0
nominal_values['Ef'] = 1.507191
nominal_values['Vm'] = 1.05
nominal_values['Vr'] = 0.5127179
nominal_values['Vis'] = -0.1507191

nominal_grid_voltage = 24e3

#
titles = {}
titles['Te'] = 'Electrical Torque'
titles['Tm'] = 'Mechanical Torque'
titles['Edq0_0'] = 'd-component of the voltage behind the transient reactance'
titles['Edq0_1'] = 'q-component of the voltage behind the transient reactance'
titles['Edq0_t_0'] = 'd-component of the voltage behind the transient reactance'
titles['Edq0_t_1'] = 'd-component of the voltage behind the transient reactance'
titles['Edq0_s_0'] = 'd-component of the voltage behind the sub-transient reactance'
titles['Edq0_s_1'] = 'd-component of the voltage behind the sub-transient reactance'
titles['Vdq0_0'] = 'd-component of the terminal voltage'
titles['Vdq0_1'] = 'q-component of the terminal voltage'
titles['Idq0_0'] = 'd-component of the terminal current'
titles['Idq0_1'] = 'q-component of the terminal current'
titles['delta'] = 'Delta'
titles['w_r'] = 'Mechanical Omega'
titles['Ef'] = 'Exiter output'
titles['Vm'] = 'Output of voltage transducer'
titles['Vr'] = 'Regulator output'
titles['Vis'] = 'Output of stablizing feedback'

#
ylabels = {}
ylabels['Te'] = 'Torque (p.u.)'
ylabels['Tm'] = 'Torque (p.u.)'
ylabels['Edq0_0'] = 'Ed_t (p.u.)'
ylabels['Edq0_1'] = 'Eq_t (p.u.)'
ylabels['Edq0_t_0'] = 'Ed_t (p.u.)'
ylabels['Edq0_t_1'] = 'Eq_t (p.u.)'
ylabels['Edq0_s_0'] = 'Ed_s (p.u.)'
ylabels['Edq0_s_1'] = 'Eq_s (p.u.)'
ylabels['Vdq0_0'] = 'Vd (p.u.)'
ylabels['Vdq0_1'] = 'Vq (p.u.)'
ylabels['Idq0_0'] = 'Id (p.u.)'
ylabels['Idq0_1'] = 'Iq (p.u.)'
ylabels['delta'] = 'Delta (rad)'
ylabels['w_r'] = 'w_r (p.u.)'
ylabels['Ef'] = 'Ef (p.u.)'
ylabels['Vm'] = 'Vm (p.u.)'
ylabels['Vr'] = 'Vr (p.u.)'
ylabels['Vis'] = 'Vr2 (p.u.)'

## Declare plot function

In [ ]:
timestep_common = 10e-6
t_begin=0
t_end=20
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)

#plot Results
width = 12
height = 4

def plot_variable(title, varname_dpsim, varname_psat, ts_dpsim, ts_psat, nominal_values, ylabels):
    fig1 = plt.figure(figsize=(width, height))
    plt.plot(ts_dpsim[varname_dpsim].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx], label='DPSim')
    plt.plot(ts_psat[varname_psat].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx], '--', label='PSAT')
    #plt.plot(ts_psat_3Order_[varname_psat].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat_3Order_[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx], '--', label='PSAT')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.ylabel(ylabels[varname_dpsim])
    plt.grid()
    plt.xlim([0, 10])
    plt.show
    
    #calculate RMSE
    nom_value = nominal_values[varname_dpsim]
    rmse = ts_dpsim[varname_dpsim].rmse(ts_dpsim[varname_dpsim].interpolate(timestep_common), ts_psat[varname_psat].interpolate(timestep_common))
    print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/nom_value*100, nom_value))
        
    return rmse/nom_value*100

def plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim, ts_psat, nominal_voltage, ylabels):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    ts_dpsim_abs_pu = TimeSeries(varname_dpsim+'_abs_pu', ts_dpsim[varname_dpsim].interpolate(timestep_common).time, ts_dpsim[varname_dpsim].interpolate(timestep_common).abs().values/nominal_voltage)
    
    fig1 = plt.figure(figsize=(width, height))
    plt.plot(ts_dpsim_abs_pu.time[begin_idx:end_idx], ts_dpsim_abs_pu.values[begin_idx:end_idx], label='DPSim')
    plt.plot(ts_psat[varname_psat].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx], '--', label='PSAT')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.show
      
     #calculate RMSE
    rmse = ts_dpsim[varname_dpsim].rmse(ts_dpsim_abs_pu, ts_psat[varname_psat].interpolate(timestep_common))
    print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))
        
    return rmse/1.0*100


def plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim, ts_psat, nominal_voltage, ylabels):
    #limit rmse to post-fault evaluation
    t_post_begin=1.1
    t_post_end=20
    begin_post_idx = int(t_post_begin/timestep_common)
    end_post_idx= int(t_post_end/timestep_common)
    
    #convert psat angle to deg and in range from
    ts_psat_deg = TimeSeries(varname_psat+'_deg', ts_psat[varname_psat].interpolate(timestep_common).time, ts_psat[varname_psat].interpolate(timestep_common).values/np.pi*180)
        
    fig2 = plt.figure(figsize=(width, height))
    plt.plot(ts_dpsim[varname_dpsim].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim[varname_dpsim].interpolate(timestep_common).phase().values[begin_idx:end_idx], label='DPSim')
    plt.plot(ts_psat_deg.time[begin_idx:end_idx], ts_psat_deg.values[begin_idx:end_idx], '--', label='PSAT')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.show
    
     #calculate RMSE
    ts_dpsim_deg_post = TimeSeries(varname_dpsim+'_deg_post', ts_dpsim[varname_dpsim].interpolate(timestep_common).time[begin_post_idx:end_post_idx], ts_dpsim[varname_dpsim].interpolate(timestep_common).phase().values[begin_post_idx:end_post_idx])
    ts_psat_deg_post = TimeSeries(varname_psat+'_deg_post', ts_psat_deg.time[begin_post_idx:end_post_idx], ts_psat_deg.values[begin_post_idx:end_post_idx])
    rmse = ts_dpsim[varname_dpsim].rmse(ts_dpsim_deg_post, ts_psat_deg_post)
    print('RMSE Post-Fault {:s}  = {:.6f} (deg)'.format(varname_dpsim, rmse))
        
    return rmse

## Validation 3th Order Model

### Electrical Torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_torque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Exiter Output

In [ ]:
varname_dpsim = "Ef"
varname_psat = "vf"

rmse_ef = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $E_q^{'}$

In [ ]:
varname_dpsim = "Edq0_1"
varname_psat = "eq_t"

rmse_eq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $V_d$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $V_q$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $I_d$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)
#plt.plot(ts_dpsim_3Order[varname_dpsim].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat_3Order[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx]- ts_dpsim_3Order[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx])
#plt.ylim([0, 0.02])

### $I_q$

In [ ]:
varname_dpsim = "Idq0_1"
varname_psat = "iq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Mechanical Omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 1.5 # %
tol_id = 7 # %
tol_deg_v1 = 0.2 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_torque<tol)
assert(rmse_ef<tol)
assert(rmse_eq<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_id)
assert(rmse_iq<tol)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)

## Validation 4th Order Model

### Electrical Torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_torque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### Exciter Output

In [ ]:
varname_dpsim = "Ef"
varname_psat = "vf"

rmse_ef = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $E_d^{'}$

In [ ]:
varname_dpsim = "Edq0_0"
varname_psat = "ed_t"

rmse_ed = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $E_q^{'}$

In [ ]:
varname_dpsim = "Edq0_1"
varname_psat = "eq_t"

rmse_eq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $V_d$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $V_q$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $I_d$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $I_q$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### Mechanical Omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 1 # %
tol_id = 6
tol_deg_v1 = 0.02 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_torque<tol)
assert(rmse_ef<tol)
assert(rmse_ed<tol)
assert(rmse_eq<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_id)
assert(rmse_iq<tol)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)

## Validation 6th Order Model (Marconato's model)

### Electrical Torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_torque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Exciter output

In [ ]:
varname_dpsim = "Ef"
varname_psat = "vf"

rmse_vf = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_d^{'}$

In [ ]:
varname_dpsim = "Edq0_t_0"
varname_psat = "ed_t"

rmse_ed = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_q^{'}$

In [ ]:
varname_dpsim = "Edq0_t_1"
varname_psat = "eq_t"

rmse_eq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_d^{''}$

In [ ]:
varname_dpsim = "Edq0_s_0"
varname_psat = "ed_s"

rmse_ed_s = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_q^{''}$

In [ ]:
varname_dpsim = "Edq0_s_1"
varname_psat = "eq_s"

rmse_eq_s = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_d$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_q$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_d$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_q$

In [ ]:
varname_dpsim = "Idq0_1"
varname_psat = "iq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Mechanical omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 1.5 #%
tol_i = 7.3
tol_deg_v1 = 0.1 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_torque<tol)
assert(rmse_ef<tol)
assert(rmse_ed<tol)
assert(rmse_eq<tol)
assert(rmse_ed_s<tol)
assert(rmse_eq_s<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_i)
assert(rmse_iq<tol_i)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)

## Validation 6th Order Model (Anderson-Fouad's model)

### Electrical Torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_torque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Exciter Output

In [ ]:
varname_dpsim = "Ef"
varname_psat = "vf"

rmse_vf = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_d$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_q$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_d$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_q$

In [ ]:
varname_dpsim = "Idq0_1"
varname_psat = "iq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Mechanical omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 2 #%
tol_i = 8
tol_deg_v1 = 0.2 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_torque<tol)
assert(rmse_vf<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_i)
assert(rmse_iq<tol_i)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)

## 2. Run C++ Examples with Governor

In [ ]:
import os
import subprocess

#%matplotlib widget

name = 'SP_ReducedOrderSG_SMIB_Fault'
options = ['SGModel=3', 'SGModel=4', 'SGModel=6a', 'SGModel=6b']

dpsim_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

path_exec = dpsim_path + '/build/dpsim/examples/cxx/'
for i in range(len(options)):
    sim = subprocess.Popen([path_exec + name, '-o', options[i], '-oWithTurbineGovernor=true',  '-oFinalTime=10'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(sim.communicate()[0].decode())

### Load DPSim Results

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os

work_dir = os.getcwd() + "/logs/"
file_name_3Order = 'SP_SynGen3Order_VBR_SMIB_Fault'
file_name_4Order = 'SP_SynGen4Order_VBR_SMIB_Fault'
file_name_6aOrder = 'SP_SynGen6aOrder_VBR_SMIB_Fault'
file_name_6bOrder = 'SP_SynGen6bOrder_VBR_SMIB_Fault'


path_logfile = work_dir + file_name_3Order + "/" + file_name_3Order + ".csv"
ts_dpsim_3Order = read_timeseries_dpsim(path_logfile)

path_logfile = work_dir + file_name_4Order + "/" + file_name_4Order + ".csv"
ts_dpsim_4Order = read_timeseries_dpsim(path_logfile)

path_logfile = work_dir + file_name_6aOrder + "/" + file_name_6aOrder + ".csv"
ts_dpsim_6aOrder = read_timeseries_dpsim(path_logfile)

path_logfile = work_dir + file_name_6bOrder + "/" + file_name_6bOrder + ".csv"
ts_dpsim_6bOrder = read_timeseries_dpsim(path_logfile)

### Load PSAT Results

In [ ]:
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/SMIB-Fault/PSAT_3OrderSyGen_withGovernor_SMIB_Fault_100mS_TS_1mS.txt'
local_file_3Order = 'reference-results/PSAT_3OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.out'
urllib.request.urlretrieve(url, local_file_3Order) 
ts_psat_3Order = read_timeseries_dpsim(local_file_3Order)

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/SMIB-Fault/PSAT_4OrderSyGen_withGovernor_SMIB_Fault_100mS_TS_1mS.txt'
local_file_4Order = 'reference-results/PSAT_4OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.out'
urllib.request.urlretrieve(url, local_file_4Order)
ts_psat_4Order = read_timeseries_dpsim(local_file_4Order)

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/SMIB-Fault/PSAT_6OrderSyGen_withGovernor_SMIB_Fault_100mS_TS_1mS.txt'
local_file_6Order = 'reference-results/PSAT_6OrderSyGen_withExciter_SMIB_Fault_100mS_TS_1mS.out'
urllib.request.urlretrieve(url, local_file_6Order)
ts_psat_6Order = read_timeseries_dpsim(local_file_6Order)

## Calculate $V_d$, $V_q$, $I_d$ and $I_q$ in PSAT

### 3th Order Model

In [ ]:
#calculate vd and vq
angle = ts_psat_3Order['delta'].values - ts_psat_3Order['theta_bus1'].values
V_bus1 = ts_psat_3Order['V_Bus1'].values
vd = V_bus1 * np.sin(angle)
ts_psat_3Order['vd'] = TimeSeries('vd', ts_psat_3Order['V_Bus1'].time, vd)
vq = V_bus1 * np.cos(angle)
ts_psat_3Order['vq'] = TimeSeries('vq', ts_psat_3Order['V_Bus1'].time, vq)

#calculate id and iq in PSAT
Xd_t = 0.299900
Xq = 1.7600
i_d = (ts_psat_3Order['eq_t'].values - ts_psat_3Order['vq'].values) / Xd_t
ts_psat_3Order['id'] = TimeSeries('id', ts_psat_3Order['eq_t'].time, i_d)
i_q = ts_psat_3Order['vd'].values / Xq
ts_psat_3Order['iq'] = TimeSeries('iq', ts_psat_3Order['vd'].time, i_q)

### 4th Order Model

In [ ]:
#calculate vd and vq
angle = ts_psat_4Order['delta'].values - ts_psat_4Order['theta_bus1'].values
V_bus1 = ts_psat_4Order['V_Bus1'].values
vd = V_bus1 * np.sin(angle)
ts_psat_4Order['vd'] = TimeSeries('vd', ts_psat_4Order['V_Bus1'].time, vd)
vq = V_bus1 * np.cos(angle)
ts_psat_4Order['vq'] = TimeSeries('vq', ts_psat_4Order['V_Bus1'].time, vq)
    
#calculate id and iq in PSAT
Xd = 0.299900
Xq = 0.650000
i_d = (ts_psat_4Order['eq_t'].values - ts_psat_4Order['vq'].values) / Xd
ts_psat_4Order['id'] = TimeSeries('id', ts_psat_4Order['eq_t'].time, i_d)
i_q = -(ts_psat_4Order['ed_t'].values - ts_psat_4Order['vd'].values) / Xq
ts_psat_4Order['iq'] = TimeSeries('iq', ts_psat_4Order['ed_t'].time, i_q)

### 6th Order Model (Marconato's model)

In [ ]:
#calculate vd and vq
angle = ts_psat_6Order['delta'].values - ts_psat_6Order['theta_bus1'].values
V_bus1 = ts_psat_6Order['V_Bus1'].values
vd = V_bus1 * np.sin(angle)
ts_psat_6Order['vd'] = TimeSeries('vd', ts_psat_6Order['V_Bus1'].time, vd)
vq = V_bus1 * np.cos(angle)
ts_psat_6Order['vq'] = TimeSeries('vq', ts_psat_6Order['V_Bus1'].time, vq)

#calculate id and iq in PSAT\n",
Xd_s = 0.2299
Xq_s = 0.2500
i_d = (ts_psat_6Order['eq_s'].values - ts_psat_6Order['vq'].values) / Xd_s
ts_psat_6Order['id'] = TimeSeries('id', ts_psat_6Order['eq_s'].time, i_d)
i_q = -(ts_psat_6Order['ed_s'].values - ts_psat_6Order['vd'].values) / Xq_s
ts_psat_6Order['iq'] = TimeSeries('iq', ts_psat_6Order['ed_s'].time, i_q)

## Nominal values

In [ ]:
#nominal values (per unit)
nominal_values = {}
nominal_values['Te'] = 0.540541
nominal_values['Tm'] = 0.540541
nominal_values['Edq0_0'] = 0.403062     #4th order model
nominal_values['Edq0_1'] = 0.944802     #3th and 4th order model
nominal_values['Edq0_t_0'] = 0.392200   #6 order model
nominal_values['Edq0_t_1'] = 0.946405   #6 order model
nominal_values['Edq0_s_0'] = 0.548309   #6 order model
nominal_values['Edq0_s_1'] = 0.918731   #6 order model
nominal_values['Vdq0_0'] = 0.639089
nominal_values['Vdq0_1'] = 0.833106
nominal_values['Idq0_0'] = 0.372443
nominal_values['Idq0_1'] = 0.363119
nominal_values['delta'] = 0.909230
nominal_values['w_r'] = 1.0
nominal_values['Ef'] = 1.507191
nominal_values['Vm'] = 1.05
nominal_values['Vr'] = 0.5127179
nominal_values['Vis'] = -0.1507191

nominal_grid_voltage = 24e3

#
titles = {}
titles['Te'] = 'Electrical Torque'
titles['Tm'] = 'Mechanical Torque'
titles['Edq0_0'] = 'd-component of the voltage behind the transient reactance'
titles['Edq0_1'] = 'q-component of the voltage behind the transient reactance'
titles['Edq0_t_0'] = 'd-component of the voltage behind the transient reactance'
titles['Edq0_t_1'] = 'd-component of the voltage behind the transient reactance'
titles['Edq0_s_0'] = 'd-component of the voltage behind the sub-transient reactance'
titles['Edq0_s_1'] = 'd-component of the voltage behind the sub-transient reactance'
titles['Vdq0_0'] = 'd-component of the terminal voltage'
titles['Vdq0_1'] = 'q-component of the terminal voltage'
titles['Idq0_0'] = 'd-component of the terminal current'
titles['Idq0_1'] = 'q-component of the terminal current'
titles['delta'] = 'Delta'
titles['w_r'] = 'Mechanical Omega'
titles['Ef'] = 'Exciter Output'
titles['Vm'] = 'Output of voltage transducer'
titles['Vr'] = 'Regulator output'
titles['Vis'] = 'Output of stablizing feedback'

#
ylabels = {}
ylabels['Te'] = 'Torque (p.u.)'
ylabels['Tm'] = 'Torque (p.u.)'
ylabels['Edq0_0'] = 'Ed_t (p.u.)'
ylabels['Edq0_1'] = 'Eq_t (p.u.)'
ylabels['Edq0_t_0'] = 'Ed_t (p.u.)'
ylabels['Edq0_t_1'] = 'Eq_t (p.u.)'
ylabels['Edq0_s_0'] = 'Ed_s (p.u.)'
ylabels['Edq0_s_1'] = 'Eq_s (p.u.)'
ylabels['Vdq0_0'] = 'Vd (p.u.)'
ylabels['Vdq0_1'] = 'Vq (p.u.)'
ylabels['Idq0_0'] = 'Id (p.u.)'
ylabels['Idq0_1'] = 'Iq (p.u.)'
ylabels['delta'] = 'Delta (rad)'
ylabels['w_r'] = 'w_r (p.u.)'
ylabels['Ef'] = 'vf (p.u.)'
ylabels['Vm'] = 'Vm (p.u.)'
ylabels['Vr'] = 'Vr (p.u.)'
ylabels['Vis'] = 'Vr2 (p.u.)'

### Declare plot function

In [ ]:
timestep_common = 10e-6
t_begin=0
t_end=20
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)

#plot Results
width = 12
height = 4

def plot_variable(title, varname_dpsim, varname_psat, ts_dpsim, ts_psat, nominal_values, ylabels):
    fig1 = plt.figure(figsize=(width, height))
    plt.plot(ts_dpsim[varname_dpsim].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx], label='DPSim')
    plt.plot(ts_psat[varname_psat].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx], '--', label='PSAT')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.ylabel(ylabels[varname_dpsim])
    plt.grid()
    plt.xlim([0, 10])
    plt.show
    
    #calculate RMSE
    nom_value = nominal_values[varname_dpsim]
    rmse = ts_dpsim[varname_dpsim].rmse(ts_dpsim[varname_dpsim].interpolate(timestep_common), ts_psat[varname_psat].interpolate(timestep_common))
    print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/nom_value*100, nom_value))
        
    return rmse/nom_value*100

def plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim, ts_psat, nominal_voltage, ylabels):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    ts_dpsim_abs_pu = TimeSeries(varname_dpsim+'_abs_pu', ts_dpsim[varname_dpsim].interpolate(timestep_common).time, ts_dpsim[varname_dpsim].interpolate(timestep_common).abs().values/nominal_voltage)
    
    fig1 = plt.figure(figsize=(width, height))
    plt.plot(ts_dpsim_abs_pu.time[begin_idx:end_idx], ts_dpsim_abs_pu.values[begin_idx:end_idx], label='DPSim')
    plt.plot(ts_psat[varname_psat].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx], '--', label='PSAT')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.show
      
     #calculate RMSE
    rmse = ts_dpsim[varname_dpsim].rmse(ts_dpsim_abs_pu, ts_psat[varname_psat].interpolate(timestep_common))
    print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))
        
    return rmse/1.0*100


def plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim, ts_psat, nominal_voltage, ylabels):
    #limit rmse to post-fault evaluation
    t_post_begin=1.1
    t_post_end=20
    begin_post_idx = int(t_post_begin/timestep_common)
    end_post_idx= int(t_post_end/timestep_common)
    
    #convert psat angle to deg and in range from
    ts_psat_deg = TimeSeries(varname_psat+'_deg', ts_psat[varname_psat].interpolate(timestep_common).time, ts_psat[varname_psat].interpolate(timestep_common).values/np.pi*180)
        
    fig2 = plt.figure(figsize=(width, height))
    plt.plot(ts_dpsim[varname_dpsim].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim[varname_dpsim].interpolate(timestep_common).phase().values[begin_idx:end_idx], label='DPSim')
    plt.plot(ts_psat_deg.time[begin_idx:end_idx], ts_psat_deg.values[begin_idx:end_idx], '--', label='PSAT')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.show
    
     #calculate RMSE
    ts_dpsim_deg_post = TimeSeries(varname_dpsim+'_deg_post', ts_dpsim[varname_dpsim].interpolate(timestep_common).time[begin_post_idx:end_post_idx], ts_dpsim[varname_dpsim].interpolate(timestep_common).phase().values[begin_post_idx:end_post_idx])
    ts_psat_deg_post = TimeSeries(varname_psat+'_deg_post', ts_psat_deg.time[begin_post_idx:end_post_idx], ts_psat_deg.values[begin_post_idx:end_post_idx])
    rmse = ts_dpsim[varname_dpsim].rmse(ts_dpsim_deg_post, ts_psat_deg_post)
    print('RMSE Post-Fault {:s}  = {:.6f} (deg)'.format(varname_dpsim, rmse))
        
    return rmse

## Validation 3th Order Model

### Electrical Torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_etorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Mechanical Torque

In [ ]:
varname_dpsim = "Tm"
varname_psat = "pm"

rmse_mtorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $E_q^{'}$

In [ ]:
varname_dpsim = "Edq0_1"
varname_psat = "eq_t"

rmse_eq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $V_d^{'}$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $V_q^{'}$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $I_d$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### $I_q$

In [ ]:
varname_dpsim = "Idq0_1"
varname_psat = "iq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Mechanical Omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_values, ylabels)

### Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_3Order, ts_psat_3Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 1.3 # %
tol_id = 6.1 # %
tol_deg_v1 = 0.2 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_etorque<tol)
assert(rmse_mtorque<tol)
assert(rmse_eq<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_id)
assert(rmse_iq<tol)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)

# Validation 4th Order Model

### Electrical Torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_etorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### Mechanical Torque

In [ ]:
varname_dpsim = "Tm"
varname_psat = "pm"

rmse_mtorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $E_d^{'}$

In [ ]:
varname_dpsim = "Edq0_0"
varname_psat = "ed_t"

rmse_ed = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $E_q^{'}$

In [ ]:
varname_dpsim = "Edq0_1"
varname_psat = "eq_t"

rmse_eq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $V_d^{'}$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $V_q^{'}$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $I_d$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### $I_q$

In [ ]:
varname_dpsim = "Idq0_1"
varname_psat = "iq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

### Mechanical Omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_values, ylabels)

## Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_4Order, ts_psat_4Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 1 # %
tol_i = 6 # %
tol_deg_v1 = 0.2 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_etorque<tol)
assert(rmse_mtorque<tol)
assert(rmse_ed<tol)
assert(rmse_eq<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_i)
assert(rmse_iq<tol_i)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)

## Validation 6th Order Model (Marconato's model)

### Electrical Torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_etorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Mechanical Torque

In [ ]:
varname_dpsim = "Tm"
varname_psat = "pm"

rmse_mtorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_d^{'}$

In [ ]:
varname_dpsim = "Edq0_t_0"
varname_psat = "ed_t"

rmse_eq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_q^{'}$

In [ ]:
varname_dpsim = "Edq0_t_1"
varname_psat = "eq_t"

rmse_eq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_d^{''}$

In [ ]:
varname_dpsim = "Edq0_s_0"
varname_psat = "ed_s"

rmse_ed_s = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $E_q^{''}$

In [ ]:
varname_dpsim = "Edq0_s_1"
varname_psat = "eq_s"

rmse_eq_s = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_d^{'}$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_q^{'}$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_d^{'}$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_q^{'}$

In [ ]:
varname_dpsim = "Idq0_1"
varname_psat = "iq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Mechanical Omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_values, ylabels)

### Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6aOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 1.2 #%
tol_i = 7.3
tol_deg_v1 = 0.2 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_etorque<tol)
assert(rmse_mtorque<tol)
assert(rmse_ed<tol)
assert(rmse_eq<tol)
assert(rmse_ed_s<tol)
assert(rmse_eq_s<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_i)
assert(rmse_iq<tol_i)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)

## Validation 6th Order Model (Anderson-Fouad's model)

### Electrical torque

In [ ]:
varname_dpsim = "Te"
varname_psat = "p"

rmse_etorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Mechanical torque

In [ ]:
varname_dpsim = "Tm"
varname_psat = "pm"

rmse_mtorque = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_d^{'}$

In [ ]:
varname_dpsim = "Vdq0_0"
varname_psat = "vd"

rmse_vd = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $V_q^{'}$

In [ ]:
varname_dpsim = "Vdq0_1"
varname_psat = "vq"

rmse_vq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_d^{'}$

In [ ]:
varname_dpsim = "Idq0_0"
varname_psat = "id"

rmse_id = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### $I_q^{'}$

In [ ]:
varname_dpsim = "Idq0_1"
varname_psat = "iq"

rmse_iq = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Delta

In [ ]:
varname_dpsim = "delta"
varname_psat = "delta"

rmse_delta = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Mechanical omega

In [ ]:
varname_dpsim = "w_r"
varname_psat = "omega"

rmse_omega = plot_variable(titles, varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_values, ylabels)

### Grid voltages

In [ ]:
varname_dpsim = "v1"
varname_psat = "V_Bus1"

rmse_v1_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v1"
varname_psat = "theta_bus1"

rmse_v1_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "V_Bus3"

rmse_v2_abs = plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

In [ ]:
varname_dpsim = "v2"
varname_psat = "theta_bus3"

rmse_v2_angle = plot_node_volt_angle(varname_dpsim, varname_psat, ts_dpsim_6bOrder, ts_psat_6Order, nominal_grid_voltage, ylabels)

### Assert

In [ ]:
tol = 2 #%
tol_i = 7.7
tol_deg_v1 = 0.2 # deg
tol_deg_v2 = 1e-12 # deg

assert(rmse_etorque<tol)
assert(rmse_mtorque<tol)
assert(rmse_vd<tol)
assert(rmse_vq<tol)
assert(rmse_id<tol_i)
assert(rmse_iq<tol_i)
assert(rmse_delta<tol)
assert(rmse_omega<tol)
assert(rmse_v1_abs<tol)
assert(rmse_v2_abs<tol)
#assert(rmse_v1_angle<tol_deg_v1)
assert(rmse_v2_angle<tol_deg_v2)